In [2]:
import requests
from datetime import datetime
import zipfile
import xml.etree.ElementTree as ET
from google.cloud import storage
from google.cloud import bigquery
import logging
import os

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./key-SA-carburants-composer.json"

In [4]:
def storage_client():
    try:
        storage_client = storage.Client()
        logging.info('Connected to storage.')
        return storage_client
    except Exception as e:
        logging.error(e)


In [5]:
storage_client = storage_client()

In [6]:
def upload_file_from(storage_client, bucket_name, destination_blob_name):
    try:
        # Download the file from the URL
        response = requests.get('https://donnees.roulez-eco.fr/opendata/instantane')
        if response.status_code != 200:
            raise Exception(f"Failed to download file from URL: https://donnees.roulez-eco.fr/opendata/instantane")

        # Upload the file to the bucket
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)
        blob.upload_from_string(response.content)

        logging.info(f"File uploaded to {destination_blob_name}.")
    except Exception as e:
        logging.error(e)

In [7]:
# !gsutil mb gs://carburants-composer

In [8]:
upload_file_from(storage_client, 'carburants-composer', 'carburants.zip')

In [9]:
!gsutil ls gs://carburants-composer

gs://carburants-composer/carburants.zip


In [ ]:
import zipfile

# Specify the path to the zip file in the bucket
zip_file_path = 'gs://carburants-composer/carburants.zip'

# Specify the path to the file you want to extract from the zip file
xml_file_path = 'gs://carburants-composer/carburants.xml'

# Specify temporary folder
temp_folder_path = 'gs://carburants-composer/temp/'

storage_client = storage.Client()

def unzip_file(storage_client, zip_file_path, temp_folder_path, xml_file_path):
    try:
        # Download the zip file from the bucket
        bucket = storage_client.bucket(zip_file_path.split('/')[2])
        blob = bucket.blob(zip_file_path.split('/')[3])
        blob.download_to_filename('temp.zip')

        # Unzip the file
        with zipfile.ZipFile('temp.zip', 'r') as zip_ref:
            zip_ref.extractall('temp')

        # Upload the XML file to the bucket
        bucket = storage_client.bucket(xml_file_path.split('/')[2])
        blob = bucket.blob(xml_file_path.split('/')[3])
        blob.upload_from_filename('temp/carburants.xml')

        logging.info(f"File uploaded to {xml_file_path}.")
    except Exception as e:
        logging.error(e)
